# Week 2: LangChain Core Concepts

## 📚 Session Overview

**Duration:** 2 hours  
**Week:** 2  
**Instructor-Led Session**

---

## 🎯 Learning Objectives

By the end of this session, you will be able to:
1. Understand the LangChain framework and its core abstractions
2. Work with Chat Models, Prompts, and Output Parsers
3. Build chains using LangChain Expression Language (LCEL)
4. Implement different types of memory for conversations
5. Compose complex workflows using chain components

---

## 📋 Prerequisites

- ✅ Completed Week 1 (LLM Fundamentals)
- ✅ Understanding of basic chatbot concepts
- ✅ LangChain installed (`pip install langchain langchain-openai`)

---

## ⏱️ Estimated Time

- Setup & Introduction: 10 minutes
- Section 1 (LangChain Intro): 25 minutes
- Section 2 (Core Components): 40 minutes
- Section 3 (LCEL & Chains): 30 minutes
- Section 4 (Memory): 20 minutes
- Wrap-up & Q&A: 5 minutes

---

## 🔧 Setup

In [ ]:
# Import required libraries
import os
from dotenv import load_dotenv

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser, PydanticOutputParser
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Standard imports
from typing import Dict, List
from pydantic import BaseModel, Field

# Load environment variables
load_dotenv()

print("✅ Setup complete!")
print(f"📦 LangChain imported successfully")

---

# Section 1: Introduction to LangChain

## What is LangChain?

LangChain is a **framework** for developing applications powered by language models.

### 🤔 Why LangChain?

**Without LangChain** (Week 1 approach):
- Manual message management
- Custom prompt formatting
- Manual chain creation
- No standardized interfaces

**With LangChain**:
- ✅ Standardized components
- ✅ Easy chain composition
- ✅ Built-in memory management
- ✅ Reusable prompt templates
- ✅ Rich ecosystem of integrations

---

## Core Abstractions

LangChain has several key components:

### 1. **Models**
- Chat Models (ChatOpenAI, ChatAnthropic)
- LLMs (OpenAI, HuggingFace)
- Embedding Models

### 2. **Prompts**
- Prompt Templates
- Chat Prompt Templates
- Few-shot examples

### 3. **Output Parsers**
- String Parser
- JSON Parser
- Pydantic Parser (structured outputs)

### 4. **Chains**
- Simple chains
- Sequential chains
- LCEL (LangChain Expression Language)

### 5. **Memory**
- Conversation Buffer Memory
- Conversation Summary Memory
- Custom memory implementations

---

## 1.1: Your First LangChain Model

In [ ]:
# Initialize a chat model
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.7,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

# Simple invocation
response = llm.invoke("What is LangChain?")

print("🤖 Response:")
print(response.content)
print()
print("📊 Response Type:", type(response))
print("📊 Content Type:", type(response.content))

### Understanding Messages

In [ ]:
# LangChain uses message objects
messages = [
    SystemMessage(content="You are a helpful AI assistant."),
    HumanMessage(content="Explain quantum computing in simple terms."),
]

response = llm.invoke(messages)
print("🤖 Response:")
print(response.content)

# You can also add AI messages
messages.append(AIMessage(content=response.content))
messages.append(HumanMessage(content="Give me a real-world example."))

response = llm.invoke(messages)
print("\n🤖 Follow-up Response:")
print(response.content)

---

# Section 2: Core Components Deep Dive

Let's explore each core component in detail.

## 2.1: Prompt Templates

Prompt templates make it easy to create reusable prompts with variables.

In [ ]:
# Simple prompt template
from langchain_core.prompts import PromptTemplate

# Create a template with variables
template = """You are a {role}.
Please {task} about {topic}.
Keep your response under {word_limit} words."""

prompt = PromptTemplate(
    template=template,
    input_variables=["role", "task", "topic", "word_limit"]
)

# Format the prompt
formatted_prompt = prompt.format(
    role="science teacher",
    task="explain",
    topic="photosynthesis",
    word_limit=50
)

print("📝 Formatted Prompt:")
print(formatted_prompt)
print("\n" + "="*60 + "\n")

# Use with LLM
response = llm.invoke(formatted_prompt)
print("🤖 Response:")
print(response.content)

### Chat Prompt Templates

For chat models, use `ChatPromptTemplate`:

In [ ]:
# Chat prompt template with multiple messages
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a {personality} assistant."),
    ("human", "Hi, my name is {name}."),
    ("ai", "Hello {name}! Nice to meet you. How can I help you today?"),
    ("human", "{user_input}")
])

# Format the messages
messages = chat_prompt.format_messages(
    personality="friendly",
    name="Alex",
    user_input="Tell me about yourself."
)

print("📝 Formatted Messages:")
for msg in messages:
    print(f"[{msg.__class__.__name__}]: {msg.content}")
print("\n" + "="*60 + "\n")

# Use with LLM
response = llm.invoke(messages)
print("🤖 Response:")
print(response.content)

### ✏️ Try It Yourself!

**Exercise:** Create a prompt template for a translation task.

In [ ]:
# YOUR CODE HERE
# Create a translation prompt template with variables:
# - source_language
# - target_language
# - text


## 2.2: Output Parsers

Output parsers help structure the LLM's response.

### String Output Parser

In [ ]:
# String output parser (most common)
str_parser = StrOutputParser()

# Without parser
response_without_parser = llm.invoke("Say hello")
print("Without Parser:")
print(f"Type: {type(response_without_parser)}")
print(f"Content: {response_without_parser.content}")
print()

# With parser
response_with_parser = str_parser.invoke(response_without_parser)
print("With Parser:")
print(f"Type: {type(response_with_parser)}")
print(f"Content: {response_with_parser}")

### JSON Output Parser

In [ ]:
# JSON output parser for structured data
json_parser = JsonOutputParser()

# Create a prompt that asks for JSON
json_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Always respond with valid JSON."),
    ("human", """Analyze the sentiment of this text and return a JSON object with:
    - sentiment: positive, negative, or neutral
    - confidence: a number from 0 to 1
    - key_phrases: list of important phrases
    
    Text: {text}
    
    Return only the JSON, no other text.""")
])

# Format and invoke
messages = json_prompt.format_messages(text="I absolutely love this product! It exceeded all my expectations.")
response = llm.invoke(messages)

print("Raw Response:")
print(response.content)
print("\n" + "="*60 + "\n")

# Parse JSON
try:
    parsed = json_parser.parse(response.content)
    print("Parsed JSON:")
    print(f"Type: {type(parsed)}")
    print(f"Sentiment: {parsed.get('sentiment')}")
    print(f"Confidence: {parsed.get('confidence')}")
    print(f"Key Phrases: {parsed.get('key_phrases')}")
except Exception as e:
    print(f"❌ Parsing error: {e}")

### Pydantic Output Parser (Structured Outputs)

In [ ]:
from langchain_core.output_parsers import PydanticOutputParser

# Define the structure using Pydantic
class PersonInfo(BaseModel):
    name: str = Field(description="Person's full name")
    age: int = Field(description="Person's age")
    occupation: str = Field(description="Person's job or occupation")
    hobbies: List[str] = Field(description="List of hobbies")

# Create parser
pydantic_parser = PydanticOutputParser(pydantic_object=PersonInfo)

# Create prompt with format instructions
pydantic_prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract person information from the text."),
    ("human", """Extract the person's information from this text:
    
    {text}
    
    {format_instructions}""")
])

# Get format instructions
format_instructions = pydantic_parser.get_format_instructions()

# Format prompt
messages = pydantic_prompt.format_messages(
    text="John Smith is a 32-year-old software engineer who enjoys hiking, photography, and playing guitar.",
    format_instructions=format_instructions
)

# Get response and parse
response = llm.invoke(messages)
person = pydantic_parser.parse(response.content)

print("✅ Parsed Person Information:")
print(f"Type: {type(person)}")
print(f"Name: {person.name}")
print(f"Age: {person.age}")
print(f"Occupation: {person.occupation}")
print(f"Hobbies: {', '.join(person.hobbies)}")

---

# Section 3: LCEL & Chain Composition

**LCEL (LangChain Expression Language)** is a declarative way to compose chains.

**Theory:**
LangChain Expression Language (LCEL) uses the pipe operator `|` to compose components into chains. This is inspired by Unix pipes where the output of one command becomes the input to the next.

**How it works:**
```python
chain = component1 | component2 | component3
result = chain.invoke(input)
```

Is equivalent to:
```python
step1 = component1(input)
step2 = component2(step1)
result = component3(step2)
```

**Why it matters:**
- Readability: Clear data flow from left to right
- Composability: Easy to add/remove/reorder steps
- Declarative: Describe what to do, not how
- Debugging: Each component can be tested independently

## Key Concept: The Pipe Operator (`|`)

LCEL uses the pipe operator to chain components:
```python
chain = prompt | model | parser
```

This is equivalent to:
```python
result = parser(model(prompt(input)))
```

## 3.1: Simple Chain

In [ ]:
# Create a simple chain: prompt -> model -> parser
simple_prompt = ChatPromptTemplate.from_template(
    "Tell me a {length} joke about {topic}."
)

# Compose the chain
simple_chain = simple_prompt | llm | StrOutputParser()

# Invoke the chain
result = simple_chain.invoke({
    "length": "short",
    "topic": "programming"
})

print("🤖 Result:")
print(result)
print("\n📊 Type:", type(result))  # Now it's a string!

## 3.2: Sequential Chain

Chains can be composed sequentially for multi-step workflows.

In [ ]:
# Step 1: Generate a topic
topic_prompt = ChatPromptTemplate.from_template(
    "Suggest a creative topic for a {genre} story. Return only the topic, one sentence."
)
topic_chain = topic_prompt | llm | StrOutputParser()

# Step 2: Write the story
story_prompt = ChatPromptTemplate.from_template(
    "Write a short {genre} story (3-4 sentences) about: {topic}"
)
story_chain = story_prompt | llm | StrOutputParser()

# Step 3: Analyze the story
analysis_prompt = ChatPromptTemplate.from_template(
    "Analyze this story and provide: mood, key themes, and a rating (1-10).\n\nStory: {story}"
)
analysis_chain = analysis_prompt | llm | StrOutputParser()

# Execute the pipeline
genre = "science fiction"

print("📖 Generating story...\n")

# Step 1
topic = topic_chain.invoke({"genre": genre})
print(f"📌 Topic: {topic}")
print()

# Step 2
story = story_chain.invoke({"genre": genre, "topic": topic})
print(f"📝 Story:\n{story}")
print()

# Step 3
analysis = analysis_chain.invoke({"story": story})
print(f"🔍 Analysis:\n{analysis}")

## 3.3: RunnablePassthrough

**Theory:**
`RunnablePassthrough` allows data to flow through a chain while also being used by other components. Think of it as a "splitter" that keeps the original data while also processing it.

**Two main uses:**

1. **Pass data unchanged:**
```python
RunnablePassthrough()  # Input passes through as-is
```

2. **Assign new fields to dict:**
```python
RunnablePassthrough.assign(
    new_field=some_chain
)
```

**Example flow:**
```python
Input: {"text": "Hello"}
↓
RunnablePassthrough.assign(translation=translate_chain)
↓
Output: {"text": "Hello", "translation": "Hola"}
```

**Why it matters:**
- Preserve context: Keep original data while adding new information
- Build complex objects: Gradually construct result dictionaries
- Parallel operations: Run multiple chains on the same input

In [ ]:
# Example: Translate and also keep the original
translate_prompt = ChatPromptTemplate.from_template(
    "Translate this to {language}: {text}"
)

# Chain that returns both original and translation
translation_chain = (
    RunnablePassthrough.assign(
        translation=(translate_prompt | llm | StrOutputParser())
    )
)

result = translation_chain.invoke({
    "text": "Hello, how are you?",
    "language": "Spanish"
})

print("📝 Result:")
print(f"Original: {result['text']}")
print(f"Translation: {result['translation']}")

## 3.4: RunnableLambda

**Theory:**
`RunnableLambda` lets you use custom Python functions inside chains. This bridges LangChain components with your own logic.

**When to use:**
- Custom data transformation
- API calls to external services
- Data validation
- Format conversion
- Business logic

**Example:**
```python
def word_count(text: str) -> dict:
    return {"text": text, "count": len(text.split())}

chain = prompt | llm | StrOutputParser() | RunnableLambda(word_count)
```

**Why it matters:**
- Flexibility: Integrate any Python code
- No limitations: Not restricted to LangChain components
- Reusability: Use existing functions in chains
- Testing: Easy to test functions independently

In [ ]:
# Custom function to process text
def word_count(text: str) -> Dict:
    """Count words in text."""
    words = text.split()
    return {
        "text": text,
        "word_count": len(words),
        "char_count": len(text)
    }

# Create chain with custom function
summary_prompt = ChatPromptTemplate.from_template(
    "Summarize this in one sentence: {text}"
)

summary_chain = (
    summary_prompt 
    | llm 
    | StrOutputParser() 
    | RunnableLambda(word_count)  # Apply custom function
)

result = summary_chain.invoke({
    "text": """Artificial intelligence is transforming industries by automating tasks, 
    providing insights from data, and enabling new capabilities. Machine learning, 
    a subset of AI, allows systems to learn from data without explicit programming."""
})

print("📊 Summary Analysis:")
print(f"Summary: {result['text']}")
print(f"Word Count: {result['word_count']}")
print(f"Character Count: {result['char_count']}")

## 3.5: RunnableParallel

**Theory:**
`RunnableParallel` executes multiple chains simultaneously on the same input and combines results into a dictionary.

**How it works:**
```python
parallel = RunnableParallel(
    sentiment=sentiment_chain,
    summary=summary_chain,
    keywords=keywords_chain
)
result = parallel.invoke({"text": "..."})  
# → {"sentiment": "...", "summary": "...", "keywords": [...]}
```

**Visual representation:**
```
Input
  ├── Chain A → result_a
  ├── Chain B → result_b
  └── Chain C → result_c
       ↓
{"a": result_a, "b": result_b, "c": result_c}
```

**Why it matters:**
- Performance: Run independent operations concurrently
- Efficiency: Save time on parallel-safe operations
- Organization: Group related analyses together
- Comprehensive results: Get multiple perspectives at once

In [ ]:
# Define multiple chains for different analyses
sentiment_prompt = ChatPromptTemplate.from_template(
    "What is the sentiment of this text (positive/negative/neutral)? Text: {text}. Answer with just one word."
)
sentiment_chain = sentiment_prompt | llm | StrOutputParser()

language_prompt = ChatPromptTemplate.from_template(
    "What language is this text in? Text: {text}. Answer with just the language name."
)
language_chain = language_prompt | llm | StrOutputParser()

summary_prompt = ChatPromptTemplate.from_template(
    "Summarize in 5 words: {text}"
)
summary_chain = summary_prompt | llm | StrOutputParser()

# Combine chains in parallel
parallel_chain = RunnableParallel(
    sentiment=sentiment_chain,
    language=language_chain,
    summary=summary_chain
)

# Execute all chains simultaneously
result = parallel_chain.invoke({
    "text": "I love learning about artificial intelligence! It's fascinating."
})

print("🔄 Parallel Analysis Results:")
print(f"Sentiment: {result['sentiment']}")
print(f"Language: {result['language']}")
print(f"Summary: {result['summary']}")

### ✏️ Try It Yourself!

**Exercise:** Create a chain that translates text to multiple languages in parallel.

In [ ]:
# YOUR CODE HERE
# Create chains that translate to Spanish, French, and German in parallel


---

# Section 4: Memory Management

LangChain provides built-in memory classes for conversation management.

## 4.1: ConversationBufferMemory

Stores all messages in the conversation.

In [ ]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

chain = prompt | llm | StrOutputParser()

conversation = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

config = {"configurable": {"session_id": "demo_session"}}

# Have a conversation
print("Conversation 1:")
response1 = conversation.invoke({"input": "Hi, my name is Alex."}, config=config)
print(f"AI: {response1}")
print()

print("Conversation 2:")
response2 = conversation.invoke({"input": "What's my name?"}, config=config)
print(f"AI: {response2}")
print()

# View memory
print("\n📜 Memory Contents:")
history = get_session_history("demo_session")
print(f"Messages: {history.messages}")

## 4.2: ConversationBufferWindowMemory

Stores only the last K messages.

In [ ]:
class WindowedChatHistory(BaseChatMessageHistory):
    def __init__(self, k: int = 2):
        self.messages: List = []
        self.k = k
    
    def add_message(self, message):
        self.messages.append(message)
        if len(self.messages) > self.k * 2:
            self.messages = self.messages[-self.k * 2:]
    
    def clear(self):
        self.messages = []

window_store = {}

def get_windowed_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in window_store:
        window_store[session_id] = WindowedChatHistory(k=2)
    return window_store[session_id]

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

chain = prompt | llm | StrOutputParser()

conversation_window = RunnableWithMessageHistory(
    chain,
    get_windowed_history,
    input_messages_key="input",
    history_messages_key="history"
)

window_config = {"configurable": {"session_id": "window_session"}}

# Have multiple conversations
conversation_window.invoke({"input": "My favorite color is blue."}, config=window_config)
conversation_window.invoke({"input": "I have a dog named Max."}, config=window_config)
conversation_window.invoke({"input": "I work as a software engineer."}, config=window_config)
conversation_window.invoke({"input": "I love hiking on weekends."}, config=window_config)

# Test memory
print("Testing memory with window size of 2...\n")
response = conversation_window.invoke({"input": "What's my favorite color?"}, config=window_config)
print(f"Question: What's my favorite color?")
print(f"AI: {response}")
print()

# View what's in memory
print("📜 Current Memory (last 2 interactions):")
history = get_windowed_history("window_session")
print(f"Number of messages: {len(history.messages)}")

## 4.3: ConversationSummaryMemory

Automatically summarizes the conversation to save tokens.

In [ ]:
class SummaryChatHistory(BaseChatMessageHistory):
    def __init__(self, llm, max_messages: int = 6):
        self.messages: List = []
        self.llm = llm
        self.max_messages = max_messages
        self.summary = ""
    
    def add_message(self, message):
        self.messages.append(message)
        if len(self.messages) > self.max_messages:
            self._summarize_and_trim()
    
    def _summarize_and_trim(self):
        messages_to_summarize = self.messages[:-2]
        conversation_text = "\n".join([
            f"{m.type}: {m.content}" for m in messages_to_summarize
        ])
        
        summary_prompt = f"""Summarize the following conversation concisely:

{conversation_text}

Summary:"""
        
        summary_response = self.llm.invoke(summary_prompt)
        self.summary = summary_response.content
        self.messages = self.messages[-2:]
    
    def clear(self):
        self.messages = []
        self.summary = ""

summary_store = {}

def get_summary_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in summary_store:
        summary_store[session_id] = SummaryChatHistory(llm)
    return summary_store[session_id]

prompt_with_summary = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Previous conversation summary: {summary}"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

def add_summary(inputs):
    history = get_summary_history("summary_session")
    return {
        "input": inputs["input"],
        "history": history.messages,
        "summary": history.summary or "No previous conversation"
    }

chain = (
    RunnableLambda(add_summary)
    | prompt_with_summary
    | llm
    | StrOutputParser()
)

conversation_summary = RunnableWithMessageHistory(
    chain,
    get_summary_history,
    input_messages_key="input",
    history_messages_key="history"
)

summary_config = {"configurable": {"session_id": "summary_session"}}

# Have a longer conversation
conversation_summary.invoke({"input": "Hi! I'm learning about AI and machine learning."}, config=summary_config)
conversation_summary.invoke({"input": "I'm particularly interested in natural language processing."}, config=summary_config)
conversation_summary.invoke({"input": "I've been working on a chatbot project using Python."}, config=summary_config)
conversation_summary.invoke({"input": "The chatbot helps users find information quickly."}, config=summary_config)

# View the summary
print("📜 Conversation Summary:")
history = get_summary_history("summary_session")
if history.summary:
    print(f"Summary: {history.summary}")
print(f"Recent messages: {len(history.messages)}")

## 4.4: Custom Memory with LCEL

In [ ]:
# Create a custom conversational chain with LCEL
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

# Store for conversation histories
store = {}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Create prompt with message placeholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

# Create chain
chain = prompt | llm | StrOutputParser()

# Wrap with message history
conversational_chain = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

# Have a conversation
config = {"configurable": {"session_id": "user123"}}

response1 = conversational_chain.invoke(
    {"input": "My name is Sarah."},
    config=config
)
print(f"AI: {response1}")
print()

response2 = conversational_chain.invoke(
    {"input": "What's my name?"},
    config=config
)
print(f"AI: {response2}")

---

# 🎯 Summary & Key Takeaways

## What We Learned:

### 1. **LangChain Framework**
- Why LangChain matters for AI applications
- Core abstractions and components
- Standardized interfaces for LLMs

### 2. **Prompt Templates**
- Creating reusable prompts with variables
- ChatPromptTemplate for chat models
- MessagesPlaceholder for dynamic conversations

### 3. **Output Parsers**
- StrOutputParser for simple text
- JsonOutputParser for structured data
- PydanticOutputParser for type-safe outputs

### 4. **LCEL (LangChain Expression Language)**
- Pipe operator (`|`) for chain composition
- RunnablePassthrough for data flow
- RunnableLambda for custom functions
- RunnableParallel for concurrent execution

### 5. **Memory Management**
- ConversationBufferMemory for full history
- ConversationBufferWindowMemory for recent messages
- ConversationSummaryMemory for token efficiency
- Custom memory with RunnableWithMessageHistory

---

## 📝 Next Steps:

### Exercises for This Week:

**Exercise 1 (Due Monday):** `02_exercise_email_assistant.ipynb`
- Build multi-step email processing chain
- Implement structured outputs
- Use LCEL composition

**Exercise 2 (Due Friday):** `03_exercise_multilang_processor.ipynb`
- Create translation and analysis pipeline
- Implement parallel processing
- Add error handling

---

## 🤔 Reflection Questions:

1. How is LangChain different from direct API calls?
2. When would you use parallel chains vs sequential chains?
3. What are the trade-offs between different memory types?
4. How can LCEL improve code readability?

---

## 📚 Additional Resources:

- [LangChain Documentation](https://python.langchain.com/)
- [LCEL Guide](https://python.langchain.com/docs/expression_language/)
- [Prompt Templates](https://python.langchain.com/docs/modules/model_io/prompts/)

---

**Next Week:** RAG (Retrieval-Augmented Generation) with PGVector! 🚀